In [1]:
from openai import OpenAI
import pandas as pd
import json
from sklearn.metrics import classification_report

In [2]:
client = OpenAI(api_key='')
MODEL = 'gpt-3.5-turbo-1106'

### Task 1: Pun Detection

In [3]:
df = pd.read_json('data/llm_task1_hetero.json', orient="records")
df.head()

,id,text,label
0,het_857,Those whose child is going through a'phase'may...,1
1,het_368,Bureaucrats cut red tape-- lengthwise.,0
2,het_1204,If they say you look good in glasses that woul...,1
3,het_1043,"Nearing the end of his sentence, the convicted...",1
4,het_1098,A logger went to his doctor because of a probl...,1


In [4]:
test = df
len(test)

3

In [6]:
def prepare_prompt_humor(sentence):
  messages=[
  { "role": "system",
    "content": f'''
You are to classify whether a given sentence is a pun based on the following criteria:
1. Ambiguity -  there exists a word in the sentence that has two similarly likely interpretations.
2. Distinctiveness - the two interpretations are very different from each other i.e. how distinct are the words semantically related to the two interpretations from each other. There needs to be at least one different word in the set of words supporting each interpretation.\n
For a given sentence to be a pun, it should satisfy BOTH criteria - Ambiguity and Distinctiveness.\n
The final output is either true or false where true means that the sentence is a pun.\n
It is possible that a sentence does not have a word that is ambiguous. The result in this case is false.\n
It is possible that a sentence has Ambiguity but no Distinctiveness. The result in this case is false.\n
Here are three examples of the cases you will encounter:
1. An example with both ambiguity and distinctiveness - 
The magician got so mad that he pulled his hare out.
{{
  "output": true,
  "ambiguity": "The pun word 'hare' supports two plausible interpretations of 'hare' meaning a rabbit and 'hair' meaning human hair.",
  "distinctiveness": "In the given sentence, the words 'magician' relates to 'hare' while 'pulled' refers to the second interpretation of 'hair'. Both of these are distinct where one refers to a magician's animal while the other refers to an action done in anger which is pulling your hair."
}}
2. An example with only ambiguity -
I went to the bank.
{{
  "output": false,
  "ambiguity": "The word 'bank' does have ambiguity here where it supports two plausible interpretations of bank as in a financial institution and bank as in the banks of a river.",
  "distinctiveness": "There are no other words in the sentence that provide distinctiveness to the two interpretations, the sentence is not a pun."
}}
3. An example with neither ambiguity nor distinctiveness -
Let us go home.
{{
  "output": false,
  "ambiguity": "There is no ambiguous word in the sentence.",
  "distinctiveness": "Not applicable"
}}
Identify whether the input sentence is a pun and explain the result based on ambiguity and distinctiveness in valid JSON format. Generate a response in the form of a valid JSON object with three keys: output, ambiguity and distinctiveness.\n
'''
  },
  {
    "role": "user", "content": f'''{sentence}'''
  }
]
  return messages

In [7]:
i = 0
results = []
for _, row in test.iterrows():
  prompt = prepare_prompt_humor(row['text'])
  response = client.chat.completions.create(
  model=MODEL,
  response_format={ "type": "json_object" },
  temperature=0.2,
  messages=prompt,
  timeout=5,
  )
  out = json.loads(response.choices[0].message.content)
  out['text'] = row['text']
  results.append(out)
  print(i)
  i += 1

APITimeoutError: Request timed out.

In [25]:
results_df = pd.DataFrame(results, columns=['text', 'output', 'ambiguity', 'distinctiveness',])
results_df.to_json('outputs-task1-hetero-chatgpt.json', orient="records")
len(results_df)

445

In [26]:
results_df['output'] = results_df['output'].replace({True: 1, False: 0})

In [27]:
y_true = [int(x) for x in list(test['label'])]
y_pred = [int(x) for x in list(results_df['output'])]

In [28]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.5875    0.7402    0.6551       127
           1     0.8842    0.7925    0.8358       318

    accuracy                         0.7775       445
   macro avg     0.7359    0.7663    0.7454       445
weighted avg     0.7995    0.7775    0.7842       445



### Task 2 and 3: Pun Location and Interpretation

In [31]:
def clean_sentence(text):
  res = text.replace(" ' ' ", '" ')
  res = res.replace(" ' ", "'")
  res = res.replace(" , ", ", ")
  res = res.replace(" . ", ". ")
  res = res.replace(" .", ". ")
  res = res.replace(" ! ", "! ")
  res = res.replace(" !", "! ")
  res = res.replace(" ? ", "? ")
  res = res.replace(" ?", "? ")
  res = res.replace(" - ", "-")
  return res.strip()

In [41]:
df = pd.read_json('data/llm_task2_hetero.json', orient="records")
df['sentence'] = df['sentence'].apply(clean_sentence)
df = df.rename(columns={'sentence': 'text'})
df.head()

,text,pun_word
0,In some countries there is a lot of Manuel labor.,Manuel
1,Someone who attempts long distance fraud is kn...,phony
2,"In the delivery room, the unborn infant expect...",berth
3,Beautifully manicured lawns are highly sod after.,sod
4,"I saw something similar to moss the other day,...",lichen


In [42]:
test = df
len(test)

318

In [43]:
def prepare_prompt_humor(sentence):
  messages=[
  { "role": "system",
    "content": f'''
You are to identify the pun word in a given sentence based on the following criteria:
1. Ambiguity -  there exists a word in the sentence that has two similarly likely interpretations.
2. Distinctiveness - the two interpretations are very different from each other i.e. how distinct are the words semantically related to the two interpretations from each other. There needs to be at least one different word in the set of words supporting each interpretation.\n
For a word to be a pun word, it should satisfy BOTH criteria - Ambiguity and Distinctiveness.\n
It is guaranteed that the sentence has a pun word and hence the selected pun word must be from the sentence in the same form as it appears.\n
Also, list all the words that realize each interpretation of the pun word. If there are multiple words that support an interpretation, separate them with a semicolon.\n
Here are two examples of the cases you will encounter:
1. An example with heterographic puns (words that sound similar) - 
The magician got so mad that he pulled his hare out.
{{
  "pun_word": "hare",
  "target_word": "hair",
  "source_sense": "magician",
  "target_sense": "angry;pulled"
}}

In the above sentence, the pun word 'hare' supports two plausible interpretations of 'hare' meaning a rabbit and 'hair' meaning human hair. In the given sentence, the words 'magician' relates to 'hare' while 'angry' and 'pulled' refer to the second interpretation of 'hair'.

2. An example with homographic puns (same words with different senses) -
I used to be a banker but I lost interest.
{{
  "pun_word": "interest",
  "target_word": "interest",
  "source_sense": "banker",
  "target_sense": "used;to;be"
}}

In the above sentence, the pun word 'interest' supports two distinct interpretations of interest meaning 'financial interest' and interest meaning 'desire to do something'. The words 'banker' realize the first sense of financial interest while the words 'used', 'to', 'be' refer to the second sense.

Identify the pun word in the input sentence is a pun and list the target word and the supporting words for each sense the result based on ambiguity and distinctiveness in valid JSON format. Do not explain. Generate a response in the form of a valid JSON object with four keys: pun_word, target_word, source_sense and target_sense.\n
'''
  },
  {
    "role": "user", "content": f'''{sentence}'''
  }
]
  return messages

In [44]:
i = 0
results = []
for _, row in test.iterrows():
  prompt = prepare_prompt_humor(row['text'])
  response = client.chat.completions.create(
  model=MODEL,
  response_format={ "type": "json_object" },
  temperature=0.2,
  messages=prompt,
  timeout=5,
  )
  out = json.loads(response.choices[0].message.content)
  out['text'] = row['text']
  results.append(out)
  print(i)
  i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
results_df = pd.DataFrame(results, columns=['text', 'pun_word', 'target_word', 'source_sense', 'target_sense'])
results_df.to_json('outputs-task2-hetero-chatgpt.json', orient="records")
len(results_df)

318

In [48]:
def cprf(true, pred):
  correct = (true['pun_word'].str.strip().str.lower() == pred['pun_word'].str.strip().str.lower()).sum()
  coverage = len(pred) / len(true)
  precision = correct / len(pred)
  recall = correct / len(true)
  f1 = (2 * precision * recall) / (precision + recall)
  return coverage, precision, recall, f1

In [49]:
c, p, r, f1 = cprf(test, results_df)
print(f"Coverage: {c:.4f}")
print(f"Precision: {p:.4f}")
print(f"Recall: {r:.4f}")
print(f"F1-Score: {f1:.4f}")

Coverage: 1.0000
Precision: 0.8208
Recall: 0.8208
F1-Score: 0.8208
